# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
data = pd.read_csv('../output_data/city_weather.csv')
df = pd.DataFrame(data)
df.head()
df['Max Temp'] = df['Max Temp'].astype(float)
df.dtypes

City         object
Lat         float64
Lng         float64
Max Temp    float64
Humidity      int64
Clouds        int64
Wind        float64
Country      object
Date          int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps.
gmaps.configure(api_key=g_key)

In [7]:
# Store Lat and Lng as location and Humidity as weight
locations = df[['Lat','Lng']]
weight = df.Humidity
df = df.dropna()
# Create a Heatmap layer
figure_layout = {'width': '500px', 'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=weight,dissipating=False, max_intensity=100, point_radius = 3)
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [62]:

# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 200 and 280
narrow_df = df.loc[(df["Wind"] <= 10) & (df["Clouds"] == 0) & (df["Max Temp"] >= 260) & (df["Max Temp"] <= 280)]
narrow_df = narrow_df.dropna()
narrow_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind,Country,Date
167,Ossora,59.24,163.07,273.29,90,0,2.53,RU,1602425548
209,Kumul,42.80,93.45,279.43,50,0,1.00,CN,1602425556
351,Fuyu,45.18,124.82,278.68,64,0,3.47,CN,1602425578
367,Mil'kovo,54.72,158.62,271.92,90,0,0.47,RU,1602425580
376,Bulgan,48.81,103.53,271.88,79,0,4.59,MN,1602425582
414,Biryusinsk,55.96,97.82,267.99,96,0,2.59,RU,1602425588
516,Baykit,61.67,96.37,268.12,91,0,1.22,RU,1602425603
551,Alihe,50.57,123.72,273.19,89,0,4.34,CN,1602425608


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [63]:
hotel_df = narrow_df
hotel_df['Hotel Name'] = ''
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind,Country,Date,Hotel Name
167,Ossora,59.24,163.07,273.29,90,0,2.53,RU,1602425548,
209,Kumul,42.80,93.45,279.43,50,0,1.00,CN,1602425556,
351,Fuyu,45.18,124.82,278.68,64,0,3.47,CN,1602425578,
367,Mil'kovo,54.72,158.62,271.92,90,0,0.47,RU,1602425580,
376,Bulgan,48.81,103.53,271.88,79,0,4.59,MN,1602425582,
414,Biryusinsk,55.96,97.82,267.99,96,0,2.59,RU,1602425588,
516,Baykit,61.67,96.37,268.12,91,0,1.22,RU,1602425603,
551,Alihe,50.57,123.72,273.19,89,0,4.34,CN,1602425608,


In [68]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params = {"key": g_key, 
              "types": "hotel",
              "keyword":"hotel",
              "radius":5000, 
              "location":f"{lat},{lng}"}
    response = requests.get(base_url, params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except:
        print("Cannot find nearby hotels for " + row["City"])

Cannot find nearby hotels for Ossora
Cannot find nearby hotels for Biryusinsk
Cannot find nearby hotels for Baykit


In [69]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind,Country,Date,Hotel Name
167,Ossora,59.24,163.07,273.29,90,0,2.53,RU,1602425548,
209,Kumul,42.80,93.45,279.43,50,0,1.00,CN,1602425556,哈密环球大酒店
351,Fuyu,45.18,124.82,278.68,64,0,3.47,CN,1602425578,Meilun Business Hotel
367,Mil'kovo,54.72,158.62,271.92,90,0,0.47,RU,1602425580,"Hotel ""Geologist"""
376,Bulgan,48.81,103.53,271.88,79,0,4.59,MN,1602425582,Bulgan Hotel
414,Biryusinsk,55.96,97.82,267.99,96,0,2.59,RU,1602425588,
516,Baykit,61.67,96.37,268.12,91,0,1.22,RU,1602425603,
551,Alihe,50.57,123.72,273.19,89,0,4.34,CN,1602425608,Great Xianbei Hotel Inner Mongolia


In [70]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [71]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

## Center the output map
figure_layout = {'width': '500px', 'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig



Figure(layout=FigureLayout(height='420px'))